# Logistic Regression Baseline
This notebook builds a baseline Logistic Regression model using a ColumnTransformer preprocessing pipeline (median imputation + scaling for numeric features, most-frequent imputation + OneHotEncoding for categorical features). We run cross-validation and a holdout evaluation, then save the trained model.

In [10]:
# Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import pickle
import warnings
warnings.filterwarnings('ignore')

In [11]:
# Load train data
df = pd.read_csv('train.csv')
df.shape

(140700, 20)

In [12]:
# Drop Name if present and separate X/y
if 'Name' in df.columns:
    df = df.drop(columns=['Name'])

# Target and features
y = df['Depression']
X = df.drop(columns=['Depression'])

# Identify columns
numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()
# remove id from features if present
if 'id' in numerical_cols:
    numerical_cols.remove('id')
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

print('Numerical columns:', numerical_cols)
print('Categorical columns:', categorical_cols)

Numerical columns: ['Age', 'Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction', 'Work/Study Hours', 'Financial Stress']
Categorical columns: ['Gender', 'City', 'Working Professional or Student', 'Profession', 'Sleep Duration', 'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']


In [ ]:
# Build preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

clf = Pipeline(steps=[('preprocessor', preprocessor), ('clf', LogisticRegression(max_iter=1000))])

clf

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

In [ ]:
# Cross-validate (Stratified K-Fold) with accuracy and ROC AUC
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
acc_scores = cross_val_score(clf, X, y, cv=skf, scoring='accuracy')
roc_scores = cross_val_score(clf, X, y, cv=skf, scoring='roc_auc')
print(f'Accuracy CV: {acc_scores.mean():.4f} ± {acc_scores.std():.4f}')
print(f'ROC AUC CV: {roc_scores.mean():.4f} ± {roc_scores.std():.4f}')

NameError: name 'clf' is not defined

In [ ]:
# Holdout evaluation: train/test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
y_proba = clf.predict_proba(X_val)[:,1] if hasattr(clf, 'predict_proba') else None

print('Validation accuracy:', accuracy_score(y_val, y_pred))
if y_proba is not None:
    print('Validation ROC AUC:', roc_auc_score(y_val, y_proba))

print('Classification report:')
print(classification_report(y_val, y_pred))

NameError: name 'clf' is not defined

In [ ]:
# Save the trained pipeline to disk
with open('logistic_model.pkl', 'wb') as f:
    pickle.dump(clf, f)

print('Saved logistic_model.pkl')

NameError: name 'clf' is not defined

## Next steps
- Try regularization tuning (C) or class_weight for imbalance.
- Try feature selection or polynomial features for numeric variables.
- If OneHotEncoder expands features too much, consider Target/Ordinal encoding for high-cardinality columns.